<a href="https://colab.research.google.com/github/DaeSeokSong/MachineLearningModels/blob/main/Prediction_StockPrice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Import Pakage

In [ ]:
%cd drive/My Drive/DeepLearning/BlackUp/

# AI 학습용
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential

# 기타
import lstm, time

# Funtion

In [ ]:
def Denomalization(dataset) :
    Denormalized_data = []
    for data in dataset :
        Denormalized_window = [float(window[0]) * (float(p) + 1) for p in data]
        Denormalized_data.append(Denormalized_window)

    result = np.array(Denormalized_data)
    return result

# Load Dataset

In [ ]:
""" 예측용 데이터 불러오기 """
# 10년치 삼성전자 데이터 가져오기
filename = '10y_dataset_005930.csv'
data = pd.read_csv(filename)
data.head()

# 평균값 계산
High_prices = data['High'].values
Low_prices = data["Low"].values
Avg_prices = (High_prices + Low_prices) / 2

# Window size
seq_len = 49
total_seq_len = seq_len + 1 # 예측 기반 데이터 (seq_len)개 + 예측할 데이터 1개

result = []
for idx in range(len(Avg_prices) - total_seq_len) :
    result.append(Avg_prices[idx: idx + total_seq_len])

""" 학습용 데이터 불러오기 """
# s%p 500 지수 데이터 불러오기, 국제 신용평가기관(미국의 스탠다드 푸어스)가 작성한 주가 지수 = 500개 대형기업(미국)의 주식을 포함한 지수
X_train, y_train, X_test, y_test = lstm.load_data('sp500.csv', 50, True) # lstm 모듈에서 load_data할 때 정규화(Normalize) 적용

# Normalize

In [ ]:
"""
ormalized_data = []
for window in result :
    Normalized_window = [(float(p) / float(window[0]) - 1) for p in window]
    Normalized_data.append(Normalized_window)

result = np.array(ormalized_data)
"""

result = np.array(result)

# 정규화를 안 하면 모델에 fit(학습)시에 loss율이 심해진다.
# 테스트셋 / 훈련셋 분류
row = int(round(result.shape[0] * 0.9))
# 훈련셋 분류 및 랜덤 셔플
train = result[:row, :]
np.random.shuffle(train)

"""
x_train = train[:, :-1] # 총 51개의 윈도우 중 50개
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
y_train = train[:, -1] # 총 51개의 윈도우 중 1개
"""

real_x_test = result[row:, :-1] # 총 50개의 윈도우 중 49개
real_x_test = np.reshape(real_x_test, (real_x_test.shape[0], real_x_test.shape[1], 1))
real_y_test = result[row:, -1] # 총 50개의 윈도우 중 1개

# Model(Supervised, 지도학습)

In [ ]:
"""
셀 스테이트(cell state) = 내부에 있는 게이트(gate)들을 통해 LSTM에서 어떤 정보를 기억하고 어떤 정보를 버릴지에 대해 추가적인 학습을 가능하게 한다. 
이를 통해 RNN이 가진, 기울기 소실 문제(Vanishing Gradient Problem, 역전파를 통해 학습할 때 학습 기울기가 너무 작아져 학습이 잘 안 되는 문제)를 해결할 수 있다.
"""

"""
# keras.models 중 모델을 순차적으로 정의하는 클래스
model = Sequential()

# LSTM 모듈 사용, 유닛 50개, 1층
model.add(LSTM(50, return_sequences=True, input_shape=(50,1)))
# LSTM 모듈 사용, 유닛 64개, 2층
model.add(LSTM(64, return_sequences=False))
# Output, 결과 산출
model.add(Dense(1, activation='linear'))
# 손실함수 = mse, Mean Squared Error / Optimizer = rmsprop
model.compile(loss='mse', optimizer='rmsprop')
# 개요 출력
model.summary()
"""
# keras.models 중 모델을 순차적으로 정의하는 클래스
model = Sequential()

model.add(LSTM(
    1, # 해당 층의 노드 개수
    input_shape=(50,1), # input_shape=?
    return_sequences=True)) # return_sequences == 각 시퀀스를 출력할지
model.add(Dropout(0.2)) # 과적합 방지용 Dropout 20%(==0.2)

model.add(LSTM(
    100, # 노드 100개
    return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(1, activation='sigmoid')) # 활성화(Activation)함수 = sigmoid
model.compile(loss='mse', optimizer='rmsprop')

model.summary()

# Training

In [ ]:
"""
model.fit(x_train, y_train, # 훈련셋
          validation_data=(x_test, y_test), # 테스트셋, validation_data
          batch_size=10, # 한 번에 몇 개씩 묶어서 학습시킬지, batch_size
          epochs=20) # N번 동안 반복해라, epochs
"""

# trainSet이나 testSet에 null값 섞여있으면 loss(손실률)이 nan으로 나오니 주의
model.fit(X_train, y_train, # 훈련셋
          validation_data=(X_test, y_test), # 테스트셋
          batch_size=64, # 한 번에 몇 개씩 묶어서 학습시킬지, batch_size
          epochs=8) # N번 동안 반복해라, epochs 

# Prediction

In [ ]:
# 모델 예측
pred = lstm.predict_sequences_multiple(model, real_x_test, 50, len(real_x_test.reshape(-1))) # result의 차원이 파라미터에 넣어야할 차원 보다 높아서 에러 나는 중
lstm.plot_results_multiple(pred, real_y_test, 50)

# CopyRight

In [ ]:
# 참조 사이트: https://github.com/llSourcell/How-to-Predict-Stock-Prices-Easily-Demo